# Quantum SVM

This script is used to evaluate different configurations of the QSVM over the same Iris dataset train/test split.

## Importing

In [ ]:
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua.input import ClassificationInput
from qiskit.aqua import run_algorithm, QuantumInstance
from qiskit.aqua.algorithms import QSVM
import numpy as np
import matplotlib.pyplot as plt

## Loading Real Backend

In [ ]:
#setting up account 
from qiskit import IBMQ
account_token = ''
login_address = 'https://api.quantum-computing.ibm.com/api/Hubs/ibm-q/Groups/open/Projects/main'
#IBMQ.enable_account(account_token, login_address)

## Loading Iris Dataset

In [ ]:
from DatasetLoader import LoadDataset

## QSVM

In [ ]:
from qiskit.aqua.components.feature_maps import SecondOrderExpansion, FirstOrderExpansion, PauliExpansion, self_product

features = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
train_set, test_set = LoadDataset('../dataset/Iris_training.csv', '../dataset/Iris_testing.csv', features, label='Species')

temp = [test_set[k] for k in test_set]
total_array = np.concatenate(temp)
algo_input = ClassificationInput(train_set, test_set, total_array)


In [ ]:
#this method runs aqua with all possible combination of feature maps and multi-class expansions from a given config
def run_aqua(algo_input, real_backend = False, nshots = 1024):
    # This json object sets the basic config for the QSVM algorithm object. 
    # Since programmatic object themselves are not yet fully implemented in Aqua, 
    # this is the only possible way to define algorithm objects.
    aqua_dict = {
        'problem': {'name': 'classification'},
        'algorithm': {'name': 'QSVM'},
        'backend': {},
        'multiclass_extension': {},
        'feature_map': {}
    }
    
    # This part defines backend
    if real_backend:
        aqua_dict['backend'] = {'provider': 'qiskit.IBMQ', 'name': 'ibmq_16_melbourne', 'shots': nshots}
    else:
        aqua_dict['backend'] =  {'provider': 'qiskit.BasicAer', 'name': 'qasm_simulator', 'shots': nshots}
        
    # list of all extensions and feature maps.
    
    extensions = [
        {'name': 'OneAgainstRest'},
        {'name': 'AllPairs'}, 
        {'name': 'ErrorCorrectingCode', 'code_size': 5}]
        
    feature_maps = [
        {'name': 'PauliExpansion', 'depth': 2, 'paulis':['Z', 'ZZ'], 'entanglement': 'full'},
        {'name': 'RawFeatureVector', 'feature_dimension': 4},
        {'name': 'PauliZExpansion', 'depth': 2, 'z_order': 2, 'entanglement': 'full'},
        {'name': 'SecondOrderExpansion', 'depth': 2, 'entanglement': 'full'},
        {'name': 'FirstOrderExpansion', 'depth': 2}]
        
    # looping over them
    for extension in extensions:
        for feature_map in feature_maps:
            aqua_dict['multiclass_extension'] = extension
            aqua_dict['feature_map'] = feature_map
            result = run_algorithm(aqua_dict, algo_input)
            print("\n----- Using multiclass extension: '{}' -----\n".format(extension['name']))
            print("\n----- Using feature map: '{}' -----\n".format(feature_map['name']))
            for k,v in result.items():
                print("'{}' : {}".format(k, v))


In [ ]:
%%time
run_aqua(algo_input)